In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
import os

In [2]:
(x_train, y_train), (x_test,y_test) = mnist.load_data()



In [3]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=3)


Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 646us/step - accuracy: 0.8208 - loss: 6.8187
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 627us/step - accuracy: 0.9022 - loss: 0.4106
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 607us/step - accuracy: 0.9238 - loss: 0.3063


In [5]:
# Save model

model.save("mnist_model.h5")

In [6]:
_pred_test = model.predict(x_test[:20])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [7]:
import numpy as np
for temp in _pred_test:
    print(temp)
    print(int(np.argmax(temp)))


[4.8122774e-06 1.7347155e-07 2.0257412e-05 5.2199580e-06 3.3828595e-09
 1.1122938e-09 1.4955047e-13 9.9954504e-01 2.0088824e-07 4.2422157e-04]
7
[8.5211165e-23 2.2804103e-09 1.0000000e+00 4.7444763e-13 2.7887092e-20
 8.6625935e-18 7.1474411e-31 7.8410076e-24 1.7362494e-15 0.0000000e+00]
2
[7.2589756e-09 9.9952018e-01 4.7161267e-04 3.5823755e-12 4.7581872e-11
 2.2766396e-24 4.2894954e-09 8.1520393e-06 5.9527316e-08 4.1975159e-11]
1
[9.8134583e-01 1.5251955e-10 6.9329035e-03 2.3000117e-04 3.2229419e-04
 2.6759927e-04 1.0045640e-02 5.7479017e-05 6.7590532e-04 1.2235918e-04]
0
[2.2931025e-07 4.8363763e-10 1.6976279e-13 6.2378116e-09 9.9386853e-01
 1.5227397e-13 2.3066704e-07 2.4057974e-03 8.5753876e-11 3.7252053e-03]
4
[1.9410681e-05 9.9952185e-01 4.0300338e-07 8.9118929e-10 2.5320103e-06
 1.3819219e-19 4.2497160e-04 2.5541347e-05 5.0150684e-06 3.6929018e-07]
1
[1.7671951e-08 4.4349153e-11 1.8317155e-13 3.5014214e-11 9.9997205e-01
 1.7423731e-13 2.3832419e-05 7.0427097e-07 7.9774026e-08 3.

### Now we need to create Reference data 

In [8]:
_pred_train = model.predict(x_train)
_pred_train_arry = [ int(np.argmax(x)) for x in _pred_train]
print(  np.sum(_pred_train_arry  == y_train)/len(y_train))


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 226us/step
0.9255333333333333


In [9]:
from sklearn.decomposition import PCA
import pandas as pd
def get_pca_data(_pred_result_:np.array):
    pca = PCA(n_components=5)
    features_reduced = pca.fit_transform(_pred_result_)
    return features_reduced

In [10]:
features_reduced = get_pca_data(_pred_train)
print(features_reduced)



[[-7.9900309e-02 -2.2806630e-02 -1.5288383e-02  1.5481500e-01
   1.5618327e-01]
 [-4.2299911e-02 -1.2178484e-01  4.5534555e-02  1.2974273e-01
   2.2299090e-01]
 [-7.8807183e-02 -1.7926112e-01 -2.4653831e-04  5.8409542e-01
  -5.7665151e-01]
 ...
 [-7.9900309e-02 -2.2806630e-02 -1.5288383e-02  1.5481500e-01
   1.5618327e-01]
 [-4.2819511e-02 -1.2864733e-01  1.8353392e-01  2.5502175e-01
   7.0443231e-01]
 [-3.4838358e-01  8.0669057e-01 -1.4523607e-01 -1.5428799e-01
  -5.3886969e-02]]


In [11]:
import numpy as np

def extract_image_stats(images):
    stats = {
        "mean_pixel": np.mean(images),
        "std_pixel": np.std(images),
        "min_pixel": np.min(images),
        "max_pixel": np.max(images),
    }
    return stats

train_image_data = extract_image_stats(x_train)
train_image_data

{'mean_pixel': np.float64(33.318421449829934),
 'std_pixel': np.float64(78.56748998339798),
 'min_pixel': np.uint8(0),
 'max_pixel': np.uint8(255)}

In [12]:
df_reference = pd.DataFrame(features_reduced, columns=[f'pca_{i}' for i in range(5)])
df_reference['target'] = y_train  # true labels for the test set
df_reference['timestamp'] = pd.to_datetime('now')
df_reference['mean_pixel'] = train_image_data['mean_pixel']
df_reference['std_pixel'] = train_image_data['std_pixel']
df_reference['min_pixel'] = train_image_data['min_pixel']
df_reference['max_pixel'] = train_image_data['max_pixel']


df_reference

,pca_0,pca_1,pca_2,pca_3,pca_4,target,timestamp,mean_pixel,std_pixel,min_pixel,max_pixel
0,-0.079900,-0.022807,-0.015288,0.154815,0.156183,5,2025-04-11 22:50:16.008235,33.318421,78.56749,0,255
1,-0.042300,-0.121785,0.045535,0.129743,0.222991,0,2025-04-11 22:50:16.008235,33.318421,78.56749,0,255
2,-0.078807,-0.179261,-0.000247,0.584095,-0.576652,4,2025-04-11 22:50:16.008235,33.318421,78.56749,0,255
3,0.911308,0.206807,-0.012764,-0.071851,-0.054462,1,2025-04-11 22:50:16.008235,33.318421,78.56749,0,255
4,-0.081020,-0.093365,-0.132588,0.126125,-0.096161,9,2025-04-11 22:50:16.008235,33.318421,78.56749,0,255
...,...,...,...,...,...,...,...,...,...,...,...
59995,-0.348394,0.806725,-0.145240,-0.154302,-0.053887,8,2025-04-11 22:50:16.008235,33.318421,78.56749,0,255
59996,-0.194371,0.372577,-0.060509,-0.041272,0.008749,3,2025-04-11 22:50:16.008235,33.318421,78.56749,0,255
59997,-0.079900,-0.022807,-0.015288,0.154815,0.156183,5,2025-04-11 22:50:16.008235,33.318421,78.56749,0,255
59998,-0.042820,-0.128647,0.183534,0.255022,0.704432,6,2025-04-11 22:50:16.008235,33.318421,78.56749,0,255


### Let's create current df

In [13]:
_pred_test = model.predict(x_test)
_pred_test_arry = [ int(np.argmax(x)) for x in _pred_test]
print(  np.sum(_pred_test_arry  == y_test)/len(y_test))


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 251us/step
0.9237


In [14]:
test_image_data = extract_image_stats(x_test)
test_image_data

{'mean_pixel': np.float64(33.791224489795916),
 'std_pixel': np.float64(79.17246322228644),
 'min_pixel': np.uint8(0),
 'max_pixel': np.uint8(255)}

In [15]:
features_reduced_new = get_pca_data(_pred_test)
df_new = pd.DataFrame(features_reduced_new, columns=[f'pca_{i}' for i in range(5)])
df_new['target'] = y_test  # true labels for the new data
df_new['timestamp'] = pd.to_datetime('now')
df_new['mean_pixel'] = test_image_data['mean_pixel']
df_new['std_pixel'] = test_image_data['std_pixel']
df_new['min_pixel'] = test_image_data['max_pixel']
df_new['max_pixel'] = test_image_data['max_pixel']



In [16]:
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric
from evidently.metric_preset import DataDriftPreset


In [17]:
from evidently.report import Report

report = Report(
    metrics=[
            ColumnDriftMetric(column_name='mean_pixel'),
            ColumnDriftMetric(column_name='std_pixel') ,
            ColumnDriftMetric(column_name='max_pixel'),
            ColumnDriftMetric(column_name='max_pixel'),
             DataDriftPreset()]
)
report.run(reference_data=df_reference, current_data=df_new)

#report.show()


/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divi

In [18]:
df_reference.to_csv("reference_data.csv", index=False)

In [20]:
import evidently
evidently.__version__

'0.6.6'